In [ ]:
# Imports
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats

plt.rcParams['figure.figsize'] = (15.0, 8.0) # set default size of plots
plt.rcParams['figure.facecolor'] = 'white'

pd.set_option('display.max_rows', None)

matplotlib.rcParams.update({'font.size': 15})

In [ ]:
TIME_RETRIEVAL_REE = 0
TIME_RETRIEVAL_TEE = 10
TIME_RETRIEVAL_REE_WASM = 0
TIME_RETRIEVAL_TEE_WASM = 13

In [ ]:
def compute_mean_std(name, time):
    df = pd.DataFrame(columns = ["Name","Time"])
    df["Name"] = name
    df["Time"] = time
    
    result = pd.DataFrame(columns = ["Mean","Std"])
    result["Mean"] = df.groupby("Name")["Time"].median() #df.groupby("Size")["Time"].apply(stats.gmean)
    result["Std"] = df.groupby("Name")["Time"].std() #df.groupby("Size")["Time"].apply(stats.gstd)
    
    return result

def formatForLatex(n, digits_round = 3):
    roundedAsString = str(round(n, digits_round))
    if "." in roundedAsString:
        roundedAsString = roundedAsString.rstrip("0")
        roundedAsString = roundedAsString.rstrip(".")
    return f"{roundedAsString}"

In [ ]:
#
## REE (Native) dataset
#

In [ ]:
# Import the dataset
ree_native_raw = pd.read_csv("../logs/polybench/ree-native.csv", names = ["Name","Time"])
ree_native_raw["Name"] = ree_native_raw["Name"].str.replace(".aot", "")

ree_native_raw

In [ ]:
# Adapt the time according the retrieval of the time
ree_native = ree_native_raw

ree_native["Time"] = ree_native["Time"] - 2*TIME_RETRIEVAL_REE/2

ree_native

In [ ]:
#
## REE (Wasm) dataset
#

In [ ]:
# Import the dataset
ree_wasm_raw = pd.read_csv("../logs/polybench/ree-wasm.csv", names = ["Name","Time"])
ree_wasm_raw["Name"] = ree_wasm_raw["Name"].str.replace(".aot", "")

ree_wasm_raw

In [ ]:
# Adapt the time according the retrieval of the time
ree_wasm = ree_wasm_raw

ree_wasm["Time"] = ree_wasm["Time"] - 2*TIME_RETRIEVAL_REE_WASM/2

ree_wasm

In [ ]:
#
## TEE dataset
#

In [ ]:
# Import the dataset
tee_raw = pd.read_csv("../logs/polybench/tee.csv", names = ["Name","Time"])
tee_raw["Name"] = tee_raw["Name"].str.replace(".aot", "")

tee_raw

In [ ]:
# Adapt the time according the retrieval of the time
tee = tee_raw

tee["Time"] = tee["Time"] - 2*TIME_RETRIEVAL_TEE_WASM/2

tee

In [ ]:
#
## Export
#

In [ ]:
# Shorten the names of the experiments
short_names = {
    "adi": "adi",
    "gramschmidt": "gra",
    "covariance": "cov",
    "correlation": "cor",
    "trmm": "trm",
    "lu": "lu",
    "symm": "sym",
    "seidel-2d": "s2d",
    "floyd-warshall": "flo",
    "2mm": "2mm",
    "mvt": "mvt",
    "deriche": "der",
    "3mm": "3mm",
    "trisolv": "tri",
    "gesummv": "ges",
    "doitgen": "doi",
    "bicg": "bic",
    "ludcmp": "lud",
    "gemver": "gev",
    "cholesky": "cho",
    "nussinov": "nus",
    "heat-3d": "h3d",
    "atax": "ata",
    "durbin": "dur",
    "syr2k": "s2k",
    "jacobi-2d": "j2d",
    "fdtd-2d": "f2d",
    "syrk": "syr",
    "jacobi-1d": "j1d",
    "gemm": "gem",
}

for k, v in short_names.items():
    ree_native["Name"] = ree_native["Name"].replace(k, v)
    ree_wasm["Name"] = ree_wasm["Name"].replace(k, v)
    tee["Name"] = tee["Name"].replace(k, v)

In [ ]:
# Normalize the results based on REE (Native) time
ree_native_normalized = pd.DataFrame()
ree_wasm_normalized = pd.DataFrame()
tee_normalized = pd.DataFrame()

for col in ree_native:
    if col == "Name":
        tee_normalized[col] = tee[col]
        ree_wasm_normalized[col] = ree_wasm[col]
        ree_native_normalized[col] = ree_native[col]
    else:
        tee_normalized[col] = tee[col] / ree_native[col]
        ree_wasm_normalized[col] = ree_wasm[col] / ree_native[col]
        ree_native_normalized[col] = 1
    
tee_normalized

In [ ]:
# Compute the mean and std
ree_native_export = pd.DataFrame()

ree_native_export = ree_native_export.append(compute_mean_std(ree_native_normalized["Name"], ree_native_normalized["Time"]))
ree_native_export

In [ ]:
# Compute the mean and std
ree_wasm_export = pd.DataFrame()

ree_wasm_export = ree_wasm_export.append(compute_mean_std(ree_wasm_normalized["Name"], ree_wasm_normalized["Time"]))
ree_wasm_export

In [ ]:
# Compute the mean and std
tee_export = pd.DataFrame()

tee_export = tee_export.append(compute_mean_std(tee_normalized["Name"], tee_normalized["Time"]))
tee_export

In [ ]:
#
## Temp visualisation
#

In [ ]:
df = pd.DataFrame({'REE native': ree_native_export["Mean"],
                   'REE WASM': ree_wasm_export["Mean"],
                   'TEE WASM': tee_export["Mean"]}, index=ree_native_export.index.values)
ax = df.plot.bar()
ax.set_ylim(0.9,3)

In [ ]:
tee_mean_of_mean = tee_export["Mean"].mean()
tee_mean_of_mean

In [ ]:
ree_wasm_mean_of_mean = ree_wasm_export["Mean"].mean()
ree_wasm_mean_of_mean

In [ ]:
# Retrieve the name of the experiment, order by TEE normalized run time
for name in tee_export.sort_values(by=["Mean"]).index:
    print(f"{name},", end = '')

In [ ]:
# Export CSV
#
# File format: experiment_name, mean, stddev

ree_wasm_export.to_csv('../logs/polybench/ree_wasm_formatted.csv')
tee_export.to_csv('../logs/polybench/tee_formatted.csv')

In [ ]:
# Export LaTeX

prefix = "Polybench"

floyd_warshall = tee_export.loc[tee_export.index == "flo"]["Mean"].values[0]
nussinov = tee_export.loc[tee_export.index == "nus"]["Mean"].values[0]

def write(file, column_name, number, digits_round = 3):
    f.write(f"\\def\\bm{prefix}{column_name}{{{formatForLatex(number, digits_round)}}}\n")

f = open('../logs/polybench/export.tex', "w")
write(f, "TeeGlobalSlowdown", tee_mean_of_mean, 2)
write(f, "ReeWasmGlobalSlowdown", ree_wasm_mean_of_mean, 2)
write(f, "FloydWarshallSlowdown", tee_export.loc[tee_export.index == "flo"]["Mean"].values[0])
write(f, "NussinovSlowdown", tee_export.loc[tee_export.index == "nus"]["Mean"].values[0])
write(f, "GlobalSlowdownPercent", (tee_mean_of_mean - 1) * 100, 1)
write(f, "FloydWarshallSlowdownPercent", (floyd_warshall - 1) * 100, 1)
write(f, "NussinovSlowdownPercent", (nussinov - 1) * 100, 1)
write(f, "FloydWarshallNussinovSlowdownPercent", (floyd_warshall - 1) * 100, 0)
f.close()